In [1]:
import argparse
import datetime
import glob
from pathlib import Path
# from test import repeat_eval_ckpt, eval_single_ckpt
# from noise import add_noise_to_weights

import numba
import logging
import os

import torch
import torch.distributed as dist
import torch.nn as nn
from tensorboardX import SummaryWriter

from pcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from pcdet.datasets import build_dataloader
from pcdet.models import build_network, model_fn_decorator
from pcdet.utils import common_utils
from train_utils.optimization import build_optimizer, build_scheduler
from train_utils.train_utils import train_model, model_save
from eval_utils import eval_utils
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

from hardware_noise.new_weight_mapping import weights_mapping as add_noise_to_weights



# from datetime import datetime
print(datetime.datetime.now())

def parse_config():
    parser = argparse.ArgumentParser(description='arg parser')
    parser.add_argument('--cfg_file', type=str, default='cfgs/kitti_models/pointpillar.yaml', \
                        help='specify the config for training')

    parser.add_argument('--batch_size', type=int, default=None, required=False, help='batch size for training')
    parser.add_argument('--epochs', type=int, default=None, required=False, help='number of epochs to train for')
    parser.add_argument('--workers', type=int, default=32, help='number of workers for dataloader')
    parser.add_argument('--extra_tag', type=str, default='default', help='extra tag for this experiment')
    parser.add_argument('--ckpt', type=str, default='checkpoint_epoch_80.pth', \
                        help='checkpoint to start from')
    parser.add_argument('--pretrained_model', type=str, default=None, help='pretrained_model')
    parser.add_argument('--launcher', choices=['none', 'pytorch', 'slurm'], default='none')
    parser.add_argument('--tcp_port', type=int, default=29051, help='tcp port for distrbuted training')
    parser.add_argument('--sync_bn', action='store_true', default=False, help='whether to use sync bn')
    parser.add_argument('--fix_random_seed', action='store_true', default=True, help='')
    parser.add_argument('--ckpt_save_interval', type=int, default=1, help='number of training epochs')
    parser.add_argument('--local_rank', type=int, default=0, help='local rank for distributed training')
    parser.add_argument('--max_ckpt_save_num', type=int, default=30, help='max number of saved checkpoint')
    parser.add_argument('--merge_all_iters_to_one_epoch', action='store_true', default=False, help='')
    parser.add_argument('--set', dest='set_cfgs', default=None, nargs=argparse.REMAINDER,
                        help='set extra config keys if needed')

    parser.add_argument('--max_waiting_mins', type=int, default=0, help='max waiting minutes')
    parser.add_argument('--start_epoch', type=int, default=0, help='')
    parser.add_argument('--save_to_file', action='store_true', default=False, help='')
    
    # parser.add_argument('--local_rank', type=int, default=-1, metavar='N', help='Local process rank.')

    args = parser.parse_known_args()[0]

    cfg_from_yaml_file(args.cfg_file, cfg)
    cfg.TAG = Path(args.cfg_file).stem
    cfg.EXP_GROUP_PATH = '/'.join(args.cfg_file.split('/')[1:-1])  # remove 'cfgs' and 'xxxx.yaml'

    if args.set_cfgs is not None:
        cfg_from_list(args.set_cfgs, cfg)

    return args, cfg


args, cfg = parse_config()



if args.launcher == 'none':
    dist_train = False  # True
    total_gpus = 1
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp')
    memory_gpu = [int(x.split()[2]) for x in open('tmp', 'r').readlines()]
    print('Using GPU:' + str(np.argmax(memory_gpu)))
    os.environ["CUDA_VISIBLE_DEVICES"] = str(np.argmax(memory_gpu))
    os.system('rm tmp')
else:
    args.is_master = args.local_rank == 0
    args.device = torch.cuda.device(args.local_rank)
    torch.cuda.manual_seed_all(666)
    # dist.init_process_group(backend='nccl', init_method='env://', rank = 0, world_size = 1)
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '5678'
    # dist.init_process_group(backend='nccl', init_method='env://', rank = 0, world_size = 1)
    torch.cuda.set_device(1)
    device = torch.device('cuda', cfg.LOCAL_RANK)
    total_gpus, cfg.LOCAL_RANK = getattr(common_utils, 'init_dist_%s' % args.launcher)(
        args.tcp_port, args.local_rank, backend='nccl'
    )
    dist_train = True


if args.batch_size is None:
    args.batch_size = cfg.OPTIMIZATION.BATCH_SIZE_PER_GPU
else:
    assert args.batch_size % total_gpus == 0, 'Batch size should match the number of gpus'
    args.batch_size = args.batch_size // total_gpus

args.epochs = cfg.OPTIMIZATION.NUM_EPOCHS if args.epochs is None else args.epochs

if args.fix_random_seed:
    common_utils.set_random_seed(666)

output_dir = cfg.ROOT_DIR / 'output' / cfg.EXP_GROUP_PATH / cfg.TAG / args.extra_tag
ckpt_dir = output_dir / 'ckpt'
output_dir.mkdir(parents=True, exist_ok=True)
ckpt_dir.mkdir(parents=True, exist_ok=True)

log_file = output_dir / ('log_train_%s.txt' % datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
# logger = common_utils.create_logger(log_file, rank=cfg.LOCAL_RANK)
logger = common_utils.create_logger('./baseline/pointpillar/log.txt', rank=cfg.LOCAL_RANK)

file = open('./baseline/pointpillar/result.txt','w')
file.write('results\n')
file.close()

# head = ''
# logging.basicConfig(filename='./baseline/pointpillar/log.txt',
#                     format=head)
# logger_result = logging.getLogger()
# logger_result.setLevel(logging.INFO)
# console = logging.StreamHandler()
# logging.getLogger('').addHandler(console)

# log to file
logger.info('**********************Start logging**********************')
gpu_list = os.environ['CUDA_VISIBLE_DEVICES'] if 'CUDA_VISIBLE_DEVICES' in os.environ.keys() else 'ALL'
logger.info('CUDA_VISIBLE_DEVICES=%s' % gpu_list)

if dist_train:
    logger.info('total_batch_size: %d' % (total_gpus * args.batch_size))


for key, val in vars(args).items():
    logger.info('{:16} {}'.format(key, val))
log_config_to_file(cfg, logger=logger)
if cfg.LOCAL_RANK == 0:
    os.system('cp %s %s' % (args.cfg_file, output_dir))

tb_log = SummaryWriter(log_dir=str(output_dir / 'tensorboard')) if cfg.LOCAL_RANK == 0 else None

# -----------------------create dataloader & network & optimizer---------------------------
train_set, train_loader, train_sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=args.batch_size,
    dist=dist_train, workers=args.workers,
    logger=logger,
    training=True,
    merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch,
    total_epochs=args.epochs
)

model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=train_set)
if args.sync_bn:
    model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
model.cuda()

# optimizer = build_optimizer(model, cfg.OPTIMIZATION)

# # load checkpoint if it is possible
# start_epoch = it = 0
# last_epoch = -1
# if args.pretrained_model is not None:
#     model.load_params_from_file(filename=args.pretrained_model, to_cpu=dist, logger=logger)

# if args.ckpt is not None:
#     it, start_epoch = model.load_params_with_optimizer(args.ckpt, to_cpu=dist, optimizer=optimizer, logger=logger)
#     last_epoch = start_epoch + 1
# else:
#     ckpt_list = glob.glob(str(ckpt_dir / '*checkpoint_epoch_*.pth'))
#     if len(ckpt_list) > 0:
#         ckpt_list.sort(key=os.path.getmtime)
#         it, start_epoch = model.load_params_with_optimizer(
#             ckpt_list[-1], to_cpu=dist, optimizer=optimizer, logger=logger
#         )
#         last_epoch = start_epoch + 1

# model.train()  # before wrap to DistributedDataParallel to support fixed some parameters
if dist_train:
    model = nn.parallel.DistributedDataParallel(model, device_ids=[cfg.LOCAL_RANK % torch.cuda.device_count()])
logger.info(model)

# lr_scheduler, lr_warmup_scheduler = build_scheduler(
#     optimizer, total_iters_each_epoch=len(train_loader), total_epochs=args.epochs,
#     last_epoch=last_epoch, optim_cfg=cfg.OPTIMIZATION
# )

# # -----------------------start training---------------------------
# logger.info('**********************Start training %s/%s(%s)**********************'
#             % (cfg.EXP_GROUP_PATH, cfg.TAG, args.extra_tag))
# train_model(
#     model,
#     optimizer,
#     train_loader,
#     model_func=model_fn_decorator(),
#     lr_scheduler=lr_scheduler,
#     optim_cfg=cfg.OPTIMIZATION,
#     start_epoch=start_epoch,
#     total_epochs=args.epochs,
#     start_iter=it,
#     rank=cfg.LOCAL_RANK,
#     tb_log=tb_log,
#     ckpt_save_dir=ckpt_dir,
#     train_sampler=train_sampler,
#     lr_warmup_scheduler=lr_warmup_scheduler,
#     ckpt_save_interval=args.ckpt_save_interval,
#     max_ckpt_save_num=args.max_ckpt_save_num,
#     merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch
# )

# logger.info('**********************End training %s/%s(%s)**********************\n\n\n'
#             % (cfg.EXP_GROUP_PATH, cfg.TAG, args.extra_tag))






logger.info('**********************Start evaluation %s/%s(%s)**********************' %
            (cfg.EXP_GROUP_PATH, cfg.TAG, args.extra_tag))
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=args.batch_size,
    dist=dist_train, workers=args.workers, logger=logger, training=False
)
eval_output_dir = output_dir / 'eval' / 'eval_with_train'
eval_output_dir.mkdir(parents=True, exist_ok=True)
args.start_epoch = max(args.epochs - 10, 0)  # Only evaluate the last 10 epochs

if dist_train: 
    model = model.module


logger.info('----------------Noise Experiment----------------')

save_path = './save_path/'
f = open(save_path+'3d.txt', "a+")
f.write(str(datetime.datetime.now())+'\n')
f.close()

f = open(save_path+'3d.txt', "a+")
f.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format('s', 'n', 'mthd',
                                    'Car/easy_R40',
                                    'Car/moderate_R40',
                                    'Car/hard_R40',
                                    'Pedestrian/easy_R40',
                                    'Pedestrian/moderate_R40',
                                    'Pedestrian/hard_R40',
                                    'Cyclist/easy_R40',
                                    'Cyclist/moderate_R40',
                                    'Cyclist/hard_R40',
                                    'easy_R40',
                                    'moderate_R40',
                                    'hard_R40',
                                    'avg', 'time'
                                    ))
f.close()



# import os



#     # len(hw_data_files) # 38

#     file2usability = {}
#     for f_name in sorted(hw_data_files):
#         if f_name.endswith('xlsx'):
#             print(f_name)

#             try:
#                 print(get_usability('hardware_data/'+f_name))
#                 file2usability[f_name] = get_usability('hardware_data/'+f_name)[0]
#             except:
#                 print('!!!!!!!!')
#                 file2usability[f_name] = None


hw_data_files = os.listdir('./hardware_noise/hardware_data/')

N = 10
# S = np.linspace(1e-31, 1.0, 21)#[0.3, 2.0]#
file2ap_dict = {}
sigma = 0.01
for f_name in sorted(hw_data_files)[-5:]:
    if f_name.endswith('xlsx'):
        file2ap_dict[f_name] = {}
        print(f_name)
        for n in range(N):
            model.load_params_from_file(filename=args.ckpt, logger=logger, to_cpu=dist_train)
            # add_noise_to_weights(0, s, model)
            model = add_noise_to_weights('./hardware_noise/hardware_data/'+f_name, 0, sigma, model, 'cuda')
            print('sigma:{}, evaluate-{}'.format(f_name, n))
            ret_dict = eval_utils.eval_simple(f_name, n, 
                                            cfg, model, test_loader, logger, save_path=None, dist_test=dist_train,
                                            save_to_file=args.save_to_file, result_dir=eval_output_dir
                                        )
            file2ap_dict[f_name][n] = ret_dict

logger.info('**********************End evaluation %s/%s(%s)**********************')


# logger.info('----------------Noise Experiment----------------')

# N = 10
# S = np.linspace(0., 1.5, 31)
# # print('s shape：{}'.format(S.shape))
# A = []
# E1 = np.zeros((N, S.shape[0]))
# E2 = np.zeros((N, S.shape[0]))
# E3 = np.zeros((N, S.shape[0]))
# j=0

# for s in S:
#     for n in range(N):
#         print('--------------simga: ',s,'--------------')
#         # model.load_params_from_file(filename=args.ckpt, logger=logger, to_cpu=dist_train)
#         # model.cuda()

#         add_noise_to_weights(0, s, model)
#         print('add noise')

#         ret_dict = eval_utils.eval_simple(
#                                 cfg, model, test_loader, logger, dist_test=dist_train,
#                                 save_to_file=args.save_to_file, result_dir=eval_output_dir
#                             )

#         E1[n,j] = (ret_dict['Car_3d/easy_R40'] + ret_dict['Pedestrian_3d/easy_R40'] + ret_dict['Cyclist_3d/easy_R40'])/3
#         E2[n,j] = (ret_dict['Car_3d/moderate_R40'] + ret_dict['Pedestrian_3d/moderate_R40'] + ret_dict['Cyclist_3d/moderate_R40'])/3
#         E3[n,j] = (ret_dict['Car_3d/hard_R40'] + ret_dict['Pedestrian_3d/hard_R40'] + ret_dict['Cyclist_3d/hard_R40'])/3

#         print(E1[n,j])
#         print(E2[n,j])
#         print(E3[n,j])


#     accuracy1 = np.mean(E1[:,j])
#     accuracy2 = np.mean(E2[:,j])
#     accuracy3 = np.mean(E3[:,j])
#     logger.info('sigma:{}, Accu:{}-{}-{}'.format(s,accuracy1,accuracy2,accuracy3,'3f'))
#     j = j + 1


# fig, ax = plt.subplots(1)
# ax.set_xlabel('$sigma$')
# ax.set_ylabel('Accuracy')
# ax.set_xticks([0, 0.3, 0.6, 0.9, 1.2, 1.5])
# ax.grid(True)
# ax.plot(S, np.mean(E1,0),label='easy')
# ax.plot(S, np.mean(E2,0),label='moderate')
# ax.plot(S, np.mean(E3,0),label='hard')
# ax.legend()
# ax.legend(loc='lower left', fontsize=15)
# ax.set_title("Baseline Evaluation")
# fig.savefig("./baseline/pointpillar/baseline.png", dpi=320)
# results1 = np.vstack((S, np.array(E1)))
# np.save("./baseline/pointpillar/easy.npy", results1)
# results2 = np.vstack((S, np.array(E2)))
# np.save("./baseline/pointpillar/moderate.npy", results2)
# results3 = np.vstack((S, np.array(E3)))
# np.save("./baseline/pointpillar/hard.npy", results3)

# logger.info('**********************End evaluation %s/%s(%s)**********************' %
#             (cfg.EXP_GROUP_PATH, cfg.TAG, args.extra_tag))


# if __name__ == '__main__':


2023-06-20 15:07:39,741   INFO  **********************Start logging**********************
2023-06-20 15:07:39,742   INFO  CUDA_VISIBLE_DEVICES=0
2023-06-20 15:07:39,743   INFO  cfg_file         cfgs/kitti_models/pointpillar.yaml
2023-06-20 15:07:39,743   INFO  batch_size       4
2023-06-20 15:07:39,744   INFO  epochs           80
2023-06-20 15:07:39,744   INFO  workers          32
2023-06-20 15:07:39,744   INFO  extra_tag        default
2023-06-20 15:07:39,744   INFO  ckpt             checkpoint_epoch_80.pth
2023-06-20 15:07:39,745   INFO  pretrained_model None
2023-06-20 15:07:39,745   INFO  launcher         none
2023-06-20 15:07:39,746   INFO  tcp_port         29051
2023-06-20 15:07:39,746   INFO  sync_bn          False
2023-06-20 15:07:39,746   INFO  fix_random_seed  True
2023-06-20 15:07:39,746   INFO  ckpt_save_interval 1
2023-06-20 15:07:39,747   INFO  local_rank       0
2023-06-20 15:07:39,747   INFO  max_ckpt_save_num 30
2023-06-20 15:07:39,747   INFO  merge_all_iters_to_one_ep

2023-06-20 15:07:39.678716
Using GPU:0


2023-06-20 15:07:39,890   INFO  Database filter by min points Car: 14357 => 13532
2023-06-20 15:07:39,891   INFO  Database filter by min points Pedestrian: 2207 => 2168
2023-06-20 15:07:39,891   INFO  Database filter by min points Cyclist: 734 => 705
2023-06-20 15:07:39,906   INFO  Database filter by difficulty Car: 13532 => 10759
2023-06-20 15:07:39,908   INFO  Database filter by difficulty Pedestrian: 2168 => 2075
2023-06-20 15:07:39,909   INFO  Database filter by difficulty Cyclist: 705 => 581
2023-06-20 15:07:39,913   INFO  Loading KITTI dataset
2023-06-20 15:07:39,990   INFO  Total samples for KITTI dataset: 3712
/home/pai/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2023-06-20 15:07:41,485   INFO  PointPillar(
  (vfe): PillarVFE

I-V_data_0.7um_length_200nm_diameter_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_1V_carbon_paste.xlsx
sigma:I-V_data_0.7um_length_200nm_diameter_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_1V_carbon_paste.xlsx, evaluate-0


eval:   2%|▏         | 15/943 [00:11<02:38,  5.84it/s, recall_0.3=(0, 191) / 217]

KeyboardInterrupt: 

In [5]:
file2ap_dict

{'I-V_data_2um_length_200nm_diameter_NA_third_etch_18min_Pb_ED_3h_180C_MABr_no_200nm_Ag_memory_8V_probe.xlsx': {0: 46.77617300405599,
  1: 46.38649512403802,
  2: 46.83017132502377,
  3: 46.67399199705571,
  4: 46.03440282545549,
  5: 46.56755489459731,
  6: 46.801906817900786,
  7: 46.4401838222523,
  8: 46.44166081320097,
  9: 46.304925190817066},
 'I-V_data_2um_length_250nm_diameter_NA_third_etch_15min_Pb_ED_3h_180C_MAI_no_200nm_Ag_memory_6V_carbon_paste.xlsx': {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0,
  7: 0.0,
  8: 0.0,
  9: 0.0},
 'I-V_data_2um_length_300nm_diameter_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_4V_carbon_paste.xlsx': {0: 63.20237880468791,
  1: 63.09083091144472,
  2: 63.0806417804229,
  3: 63.255072111804985,
  4: 63.20462655585953,
  5: 63.092642304637266,
  6: 63.17596881163874,
  7: 63.10885740408518,
  8: 62.98065488503734,
  9: 63.45758825852345},
 'I-V_data_2um_length_300nm_diameter_NA_third_etch_15min_Pb_ED_1h_180C_MA

In [2]:
for f_name in sorted(hw_data_files):
    print(f_name)

.ipynb_checkpoints
I-V_data_0.7um_length_200nm_diameter_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_1V_carbon_paste.xlsx
I-V_data_0.7um_length_200nm_diameter_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_1V_probe.xlsx
I-V_data_0.7um_length_200nm_diameter_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_1V_silver_paste.xlsx
I-V_data_0.7um_length_200nm_diameter_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_5V_silver_paste.xlsx
I-V_data_0.7um_length_200nm_diameter_NA_third_etch_25min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_8V_carbon_paste.xlsx
I-V_data_1.2um_length_200nm_diameter_NA_third_etch_15min_Pb_ED_10h_180+210C_FAI_PMMA_200nm_Ag_memory_6V_probe.xlsx
I-V_data_1.2um_length_200nm_diameter_NA_third_etch_15min_Pb_ED_10h_180+210C_FAI_PMMA_200nm_Ag_memory_6V_silver_paste.xlsx
I-V_data_1.2um_length_200nm_diameter_NA_third_etch_15min_Pb_ED_10h_180+210C_FAI_no_200nm_Ag_memory_6V_probe.xlsx
I-V_data_1.2um_length_200nm_diameter_NA_third_etch_15min_Pb_ED_

In [18]:
new_dict

{'I-V_data_0.7um_length_200nm_diameter_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_1V_carbon_paste.xlsx': 1.1110804232271452,
 'I-V_data_0.7um_length_200nm_diameter_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_1V_probe.xlsx': 0.2890661175740662,
 'I-V_data_0.7um_length_200nm_diameter_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_1V_silver_paste.xlsx': 0.7003371157734772,
 'I-V_data_0.7um_length_200nm_diameter_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_5V_silver_paste.xlsx': 1.186862296132586,
 'I-V_data_0.7um_length_200nm_diameter_NA_third_etch_25min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_8V_carbon_paste.xlsx': 0.6020028504994165,
 'I-V_data_1.2um_length_200nm_diameter_NA_third_etch_15min_Pb_ED_10h_180+210C_FAI_PMMA_200nm_Ag_memory_6V_probe.xlsx': 0.30952700621997337,
 'I-V_data_1.2um_length_200nm_diameter_NA_third_etch_15min_Pb_ED_10h_180+210C_FAI_PMMA_200nm_Ag_memory_6V_silver_paste.xlsx': 0.002351285159440299,
 'I-V_data_1.2um_length_2

In [20]:
# import numpy as np

# # my_dict = file2sigma  # { 'Apple': 4, 'Banana': 2, 'Orange': 6, 'Grapes': 11}
# # # 保存文件
# # np.save('file2sigma.npy', my_dict)
# # 读取文件 
# # new_dict = np.load('./hardware_noise/file2sigma.npy', allow_pickle='TRUE').item()
# # print(new_dict)

# # for i, f_name in enumerate(new_dict.keys()):
# #     # if ~f_name.endswith('xlsx'):
# #     #     pass
# #     # if f_name != sorted(hw_data_files)[i+1]:
# #     #     print('!')
# #     for u in range(10):
# #         with open('sigma.txt', 'a+') as f:
# #             print(new_dict[f_name], file=f)
            
# new_dict = np.load('./hardware_noise/file2usability.npy', allow_pickle='TRUE').item()
# # print(new_dict)

# for i, f_name in enumerate(new_dict.keys()):
#     # if ~f_name.endswith('xlsx'):
#     #     pass
#     # if f_name != sorted(hw_data_files)[i+1]:
#     #     print('!')
#     for u in range(10):
#         with open('usability.txt', 'a+') as f:
#             print(new_dict[f_name], file=f)